In [1]:
%matplotlib notebook

<hr style="border-width:3px; border-color:coral"/>
# Print Latex table (for poster)
<hr style="border-width:3px; border-color:coral"/>

In [2]:
import numpy as np
import pandas as pd

In [3]:
%reload_ext autoreload
%autoreload 2

from results_output import *

## Read in spreadsheet data

In [11]:
timing_results = pd.read_excel("Timing_results.xlsx",sheet_name=None)
div_factor = 3
df_full_hypre  = timing_results["full_hypre ({:d})".format(div_factor)]
df_schur_hypre = timing_results["schur_hypre ({:d})".format(div_factor)]

# For HYPRE results
cols_setup_hypre = ['domain_init','matrix_setup','hypre_setup','linsys_setup']
df_full_hypre['complete_setup'] = df_full_hypre[cols_setup_hypre].sum(axis=1)
df_schur_hypre['complete_setup'] = df_schur_hypre[cols_setup_hypre].sum(axis=1)

if div_factor < 4:
    # For AMGX results
    df_full_amgx   = timing_results["full_amgx ({:d})".format(div_factor)]
    if div_factor == 3:
        df_full_amgx.index = [2,3,4] # For 
    df_schur_amgx  = timing_results["schur_amgx ({:d})".format(div_factor)]

    cols_setup_amgx = ['domain_init','matrix_setup','amgx_setup','linsys_setup']
    df_full_amgx['complete_setup'] = df_full_amgx[cols_setup_amgx].sum(axis=1)
    df_schur_amgx['complete_setup'] = df_schur_amgx[cols_setup_amgx].sum(axis=1)

cols_solve_full =['linsolve']
cols_solve_schur = ['linsolve','patch_solve']

<hr style="border-width:3px; border-color:coral"/>
# Create panel of data
<hr style="border-width:3px; border-color:coral"/>

In [12]:
if div_factor < 4:
    panel = pd.Panel({'Full (HYPRE)' : df_full_hypre, 'Schur (HYPRE)' : df_schur_hypre,
                     'Full (AMGX)' : df_full_amgx, 'Schur (AMGX)' : df_schur_amgx})
else:
    panel = pd.Panel({'Full (HYPRE)' : df_full_hypre, 'Schur (HYPRE)' : df_schur_hypre})
procs = df_full_hypre['p']

## Comparisons : Solve

In [13]:
df_solve = panel.minor_xs('complete_solve')
df_solve.insert(0,'p',procs)
df_solve

,p,Full (AMGX),Full (HYPRE),Schur (AMGX),Schur (HYPRE)
0,1,NaN,68.068060,1.690548,26.907745
1,2,NaN,40.492793,1.013779,14.382487
2,4,3.168813,29.408292,0.633120,8.993995
3,8,2.435413,24.118250,0.825867,7.671255
4,16,3.466121,17.558434,0.685694,7.253857
5,32,NaN,10.271640,NaN,5.200461
6,64,NaN,8.431653,NaN,1.106002


Get data for table

In [14]:
df_its = panel.minor_xs('iterations')
df_its

,Full (AMGX),Full (HYPRE),Schur (AMGX),Schur (HYPRE)
0,NaN,27.0,19.0,17.0
1,NaN,25.0,20.0,17.0
2,53.0,26.0,20.0,16.0
3,54.0,26.0,41.0,16.0
4,100.0,25.0,43.0,16.0
5,NaN,25.0,NaN,16.0
6,NaN,25.0,NaN,16.0


In [20]:
df_table = panel.minor_xs('complete_solve')
df_table.insert(0,'p',procs)

# Insert iteration counts
if div_factor < 4:
    df_table.insert(2,'its (famgx)',df_its['Full (AMGX)'])
    df_table.insert(4,'its (fhypre)',df_its['Full (HYPRE)'])
    df_table.insert(6,'its (samgx)',df_its['Schur (AMGX)'])
    df_table.insert(8,'its (shypre)',df_its['Schur (HYPRE)'])
else:
    df_table.insert(2,'its (fhypre)', df_its['Full (HYPRE)'])
    df_table.insert(4,'its (shypre)', df_its['Schur (HYPRE)'])

df_table

,p,Full (AMGX),its (famgx),Full (HYPRE),its (fhypre),Schur (AMGX),its (samgx),Schur (HYPRE),its (shypre)
0,1,NaN,NaN,68.068060,27.0,1.690548,19.0,26.907745,17.0
1,2,NaN,NaN,40.492793,25.0,1.013779,20.0,14.382487,17.0
2,4,3.168813,53.0,29.408292,26.0,0.633120,20.0,8.993995,16.0
3,8,2.435413,54.0,24.118250,26.0,0.825867,41.0,7.671255,16.0
4,16,3.466121,100.0,17.558434,25.0,0.685694,43.0,7.253857,16.0
5,32,NaN,NaN,10.271640,25.0,NaN,NaN,5.200461,16.0
6,64,NaN,NaN,8.431653,25.0,NaN,NaN,1.106002,16.0


##  Table formatting (simple)

**Note:** This requires the BOOKTABS package.

In [16]:
if div_factor < 4:
    float_cols = ['Full (AMGX)', 'Full (HYPRE)', 'Schur (HYPRE)', 'Schur (AMGX)']
    int_cols = ['its (famgx)', 'its (fhypre)', 'its (shypre)', 'its (samgx)']
else:
    float_cols = ['Full (HYPRE)', 'Schur (HYPRE)']
    int_cols = ['its (fhypre)', 'its (shypre)']

    

fstr = {}
for cf,ci in zip(float_cols,int_cols):
    fstr = {**fstr, **{cf : '{:.2f}'.format, ci : '{:.0f}'.format}}
    
# Column format used by 'tabular', e.g. 'rrrrrrrr'
col_fmt = 'r'*df_table.shape[1]

df_latex = df_table.to_latex(formatters=fstr,escape=False,index=False, column_format=col_fmt)

# Replace NaNs
df_latex = df_latex.replace('nan','---')

# Print df_latex (a string)
print(df_latex)

\begin{tabular}{rrrrrrrrr}
\toprule
  p & Full (AMGX) & its (famgx) & Full (HYPRE) & its (fhypre) & Schur (AMGX) & its (samgx) & Schur (HYPRE) & its (shypre) \\
\midrule
  1 &         --- &         --- &        68.07 &           27 &         1.69 &          19 &         26.91 &           17 \\
  2 &         --- &         --- &        40.49 &           25 &         1.01 &          20 &         14.38 &           17 \\
  4 &        3.17 &          53 &        29.41 &           26 &         0.63 &          20 &          8.99 &           16 \\
  8 &        2.44 &          54 &        24.12 &           26 &         0.83 &          41 &          7.67 &           16 \\
 16 &        3.47 &         100 &        17.56 &           25 &         0.69 &          43 &          7.25 &           16 \\
 32 &         --- &         --- &        10.27 &           25 &          --- &         --- &          5.20 &           16 \\
 64 &         --- &         --- &         8.43 &           25 &          --- &  

## Table formatting (uses SIUNITX)

This formatting requries the SIUNITX package, but allows more flexible formatting in Latex.  Here is an example of a 
fancy multicolumn header

    \begin{tabular}{r|rr|rr|rr|rr}
    \toprule
    &  \multicolumn{2}{c}{Full (AMGX)} & \multicolumn{2}{c}{Full (HYPRE)} & \multicolumn{2}{c}{Schur (AMGX)} & \multicolumn{2}{c}    {Schur (HYPRE)} \\
    \midrule
     p & Time (s) & Iterations & Time (s) & Iterations & Time (s) & Iterations & Time (s) & Iterations \\
    \midrule
    
The header columns will have to be enclosed in `{}` to use with the siunitx package.  Example : 

    \toprule
    {p} & {Full (AMGX)} & {its (famgx)} & {Full (HYPRE)} & {its (fhypre)} & {Schur (AMGX)} & {its (samgx)} 
        &  {Schur (HYPRE)} & {its (shypre)} \\
    \midrule
    
**Note:** Formatting below requires both the **SIUNITX** and **BOOKTABS** packages

In [23]:
# Wrap headers in {} so that siunitx will not get confused
df_table_latex = df_table
for c in df_table: 
    df_table_latex.rename(columns={c : '{{{:s}}}'.format(c)}, inplace=True)

# Format using {} to get all digits in original data;  use siunitx to do the actual formatting
fstr = {}
for c in df_table_latex.columns:
    fstr = {**fstr, **{c : '{}'.format}}    # Use {} to get full set of digits in spreadsheet

# Column formatting using SIUNITX parameters
col_fmt = "\nS[zero-decimal-to-integer=true, table-format=2, table-alignment=right]\n"\
"*4{S[round-mode=places, round-precision=2, scientific-notation=fixed, \n table-alignment=right]\n"\
"S[zero-decimal-to-integer=true, table-format=5, table-alignment=right]}\n"

# Create latex string
df_latex = df_table_latex.to_latex(formatters=fstr, escape=False, index=False, column_format=col_fmt)

# Replace NaNs so they will be correctly interpreted by SIUNITX
df_latex = df_latex.replace('nan','{---}')

# Print Latex output (a string)
print(df_latex)

\begin{tabular}{
S[zero-decimal-to-integer=true, table-format=2, table-alignment=right]
*4{S[round-mode=places, round-precision=2, scientific-notation=fixed, 
 table-alignment=right]
S[zero-decimal-to-integer=true, table-format=5, table-alignment=right]}
}
\toprule
{{{p}}} & {{{Full (AMGX)}}} & {{{its (famgx)}}} & {{{Full (HYPRE)}}} & {{{its (fhypre)}}} & {{{Schur (AMGX)}}} & {{{its (samgx)}}} & {{{Schur (HYPRE)}}} & {{{its (shypre)}}} \\
\midrule
      1 &               {---} &               {---} &      68.0680601461 &               27.0 &       1.6905478806 &              19.0 &       26.9077451058 &               17.0 \\
      2 &               {---} &               {---} &      40.4927931763 &               25.0 &        1.013778522 &              20.0 &       14.3824872351 &               17.0 \\
      4 &      3.1688130379 &              53.0 &       29.408291536 &               26.0 &       0.6331196053 &              20.0 &        8.9939953395 &               16.0 \\
      8 &